In [25]:
import numpy as np
from scipy.signal import find_peaks
from typing import Optional

In [ ]:
"""Tworzenie syntetycznego sygnału EEG"""
n_channells = 4
duration = 240  # czas trwania sygnału w sekundach
s_rate = 250 # częstotliwość próbkowania
t_sample = np.arange(0,duration,1/s_rate)
n_samples = duration * s_rate

#pasma częstotliwości
freq_bands = {
    "alpha": (8,12),
    "beta": (16,30),
    "theta": (4,8),
    "delta": (0.5,4)
}

#dane eeg
eeg_data = np.zeros((n_channells,n_samples))

#generowanie sygnału
for ch in range(n_channells):
    signal = np.zeros(n_samples)
    for band,(f_min,f_max) in freq_bands.items():
        freq = np.random.uniform(f_min,f_max)
        amp = np.random.uniform(10,100)
        signal += amp * np.sin(2*np.pi*freq*t_sample)
    eeg_data[ch] = signal 
    

In [5]:
print(eeg_data.shape)

(4, 60000)


In [24]:
def count_GFP(eeg_data,distance):
    """Funkcja oblicza GFP oraz znajduje lokalne maksima GFP
    argumenty: 
    eeg_data - dane eeg (n_kanałów,n_próbek), 
    distance - odległość pomiędzy lokalnymi maksimami GFP
    zwraca: 
    peaks - dane lokalnych map szczytów GFP
    """
    if eeg_data.shape[0] > eeg_data.shape[1]: #obsługa błędu, gdy dane: (n_próbek,n_kanałów)
        raise ValueError(f"Niepoprawny kształy danych: {eeg_data.shape}\n "
                         f"Wymagana transpozycja danych (data.T)")
    GFP = np.std(eeg_data,axis=0) #axis=0 - po kolumnach
    gfp_peaks,_ = find_peaks(GFP,distance=distance) #szukanie maksim GFP
    peaks = eeg_data[:,gfp_peaks] #przypisanie indeksów gfp_peaks do wejściowych danych
    print("Poprawnie uzyskano mapy topograficzne sygnału eeg.")
    return peaks 
    



In [16]:
eeg_data_2 = eeg_data.copy().T
peaks = count_GFP(eeg_data)

Poprawnie uzyskano mapy topograficzne sygnału eeg.


In [19]:
print(peaks.shape)

(4, 8577)


In [ ]:
"""Mikrostany"""
class Microstates:
    """Klasa do analizy mikrostanów EEG.
    Klasa działa na podstawie implementacji algorytmu k-means++.
    Zaimplementowany algorytm k-means++ pozwala na grupowanie map topograficznych
    sygnału EEG w mikrostany.
    """
    def __init__(
            self,
            peaks: np.ndarray, #dane eeg z uzyskanymi mapami topograficznymi (n_channels,n_peask)
            n_microstates: int, #liczba mikrostanów do wyodrębnienia proporcjonalna liczbie klastrów (k)
            tol: float= 1e-4, #tolerancja zbieżnosci 
            ) -> None:
        self.peaks = peaks
        self.n_microstates = n_microstates
        self.tol = tol
        self.centroids = None
        self.labels = None 
    

    def centroids_initialisation(self,peaks): #funkcja inicjalizująca centroidy
        n_channels,n_peaks = peaks.shape #rozpakowanie wymiarów macierzy danych eeg
        if self.n_microstates > n_peaks: #obsługa błędu
            raise ValueError(
                f"Wybrano nieodpowiednią liczbę mikrostanów : {self.n_microstates} !\n"
                f"Liczba mikrostanów musi być większa od liczby map topograficznych w danch: {n_peaks}"
                f"Zmień liczbę"
            )

In [34]:
n_channels,n_peaks = peaks.shape
chosen_centroids = []
first_centroid = np.random.choice(n_peaks)
print(first_centroid)
chosen_centroids.append(peaks[:,first_centroid].copy())
print(chosen_centroids)






2927
[array([   1.53233314, -282.17152449,  153.62887544,   96.30146885])]


In [20]:
import matplotlib.pyplot as plt

h_bar = 1.054571817e-34  # J*s (zredukowana stała Plancka)
k_B = 1.380649e-23     # J/K (stała Boltzmanna)

# Parametry dla protonu (¹H)
gamma_H = 2.6752218744e8 # rad*s⁻¹*T⁻¹ (stała giromagnetyczna protonu)

# Funkcja do obliczania populacji
def calculate_populations(B0, T, N_total=1e6):
    """
    Oblicza populacje spinów w stanie alfa (niższa energia) i beta (wyższa energia).
    N_total to całkowita liczba spinów do normalizacji (dla lepszej wizualizacji).
    """
    delta_E = gamma_H * h_bar * B0
    ratio_beta_alpha = np.exp(-delta_E / (k_B * T))

    # N_beta / N_alpha = ratio
    # N_alpha + N_beta = N_total
    # N_alpha + N_alpha * ratio = N_total
    # N_alpha * (1 + ratio) = N_total
    N_alpha = N_total / (1 + ratio_beta_alpha)
    N_beta = N_total - N_alpha
    return N_alpha, N_beta

# Parametry symulacji
B0_tesla = 3.5          # Siła pola magnetycznego w Teslach (np. typowy spektrometr 300 MHz dla ¹H)
Temperature_K = 298.15  # Temperatura w Kelwinach (temperatura pokojowa, 25°C)

# Oblicz populacje
N_alpha, N_beta = calculate_populations(B0_tesla, Temperature_K)
population_difference = N_alpha - N_beta
excess_ppm = (population_difference / (N_alpha + N_beta)) * 1e6 # Nadwyżka w częściach na milion

print(f"Dla B0 = {B0_tesla} T i T = {Temperature_K} K:")
print(f"  Populacja N_alpha (równoległe, niższa energia): {N_alpha:.2f}")
print(f"  Populacja N_beta (antyrównoległe, wyższa energia): {N_beta:.2f}")
print(f"  Różnica populacji (N_alpha - N_beta): {population_difference:.2f}")
print(f"  Nadwyżka spinów w stanie alfa: {excess_ppm:.2f} ppm (części na milion)")

# Wizualizacja - wykres słupkowy
labels = ['N_α (równoległe)', 'N_β (antyrównoległe)']
populations = [N_alpha, N_beta]

fig, ax = plt.subplots(figsize=(8, 6))
bars = ax.bar(labels, populations, color=['skyblue', 'salmon'])
ax.set_ylabel('Względna populacja spinów (na 1 milion)')
ax.set_title(f'Rozkład Boltzmanna spinów ¹H przy B0={B0_tesla}T, T={Temperature_K}K')
ax.ticklabel_format(style='plain', axis='y') # Wyłączenie notacji naukowej dla osi Y

# Dodanie wartości na słupkach
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.005 * max(populations), f'{yval:.2f}', ha='center', va='bottom')

plt.ylim(min(populations) * 0.999, max(populations) * 1.001) # Dostosowanie granic osi Y, aby lepiej pokazać różnicę
plt.tight_layout()
plt.show()

# Wizualizacja wpływu siły pola B0 na nadwyżkę populacji
B0_values = np.linspace(1.0, 23.5, 100) # Od 1T do 23.5T (np. od 40 MHz do 1 GHz dla ¹H)
excess_populations_ppm = []
for b0_val in B0_values:
    Na, Nb = calculate_populations(b0_val, Temperature_K, N_total=1e6)
    excess_populations_ppm.append(((Na - Nb) / (Na + Nb)) * 1e6)

plt.figure(figsize=(10, 6))
plt.plot(B0_values, excess_populations_ppm, label=f'T = {Temperature_K} K')
plt.xlabel('Siła pola magnetycznego B₀ (T)')
plt.ylabel('Nadwyżka populacji spinów N_α (ppm)')
plt.title('Wpływ siły pola B₀ na nadwyżkę populacji spinów ¹H')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Wizualizacja wpływu temperatury na nadwyżkę populacji
Temperature_values = np.linspace(4, 400, 100) # Od ciekłego helu do ~120°C
excess_populations_T_ppm = []
fixed_B0 = 7.0 # Ustalona siła pola
for temp_val in Temperature_values:
    Na, Nb = calculate_populations(fixed_B0, temp_val, N_total=1e6)
    excess_populations_T_ppm.append(((Na - Nb) / (Na + Nb)) * 1e6)

plt.figure(figsize=(10, 6))
plt.plot(Temperature_values, excess_populations_T_ppm, label=f'B₀ = {fixed_B0} T')
plt.xlabel('Temperatura (K)')
plt.ylabel('Nadwyżka populacji spinów N_α (ppm)')
plt.title(f'Wpływ temperatury na nadwyżkę populacji spinów ¹H (przy B₀={fixed_B0}T)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()